<a href="https://colab.research.google.com/github/sarasabino/Accident_Rate_Project/blob/main/Src/Notebooks/00_Data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Generation

Although we have the data from the accidents occurred at the company we need to have the data from the staff with its characteristics to be able to develop a machine learning algorithm. 

The idea is to generate fake data to be able to perform the algorithm and then when data is real we can apply it.

For that purpose we are going to generate a csv file with the staff of the company. 

We are going to assume this is the static company's staff.

In [213]:
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/HS_Accidentabilidad.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/'G_Plantas y Tech'.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/Datos_plantilla_fabrica.csv .




We are going to upload the accidents data we have

In [214]:
import pandas as pd

accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')

accidentes = accidentes[(accidentes['Año']== 2018) | (accidentes['Año']==2019) | (accidentes['Año']==2020) ]

In [215]:
len(accidentes)

4180

In [216]:
accidentes['Gravedad potencial']

2057    Muy grave
2062         Leve
2063         Leve
2064         Leve
2065         Leve
          ...    
7200          NaN
7430          NaN
7436          NaN
7580          NaN
7581          NaN
Name: Gravedad potencial, Length: 4180, dtype: object

In [217]:
staff = accidentes[['Codigo accidentado', 'Puesto habitual de trabajo','Turno','Edad','Género','Planta', 'Tipo accidente', 'Tipo trabajador', 'Gravedad potencial', 'Lugar o seccion accidente']]
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente
2057,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura
2062,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos
2063,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos
2064,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado
2065,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking


In [218]:
planta = pd.read_csv('G_Plantas y Tech.csv', sep=';')
planta.drop(columns={'Activo'}, inplace=True)
planta.head()

,ID,Fabrica,Tecnología,Pais
0,1,Argentina,Blanking,Argentina
1,2,Argentina,Corte Longitudinal,Argentina
2,3,Argentina,Corte Transversal,Argentina
3,4,Arizona,Células Soldadura,EEUU
4,5,Arizona,Corte Láser,EEUU


In [219]:
len(staff)

4180

In [220]:
staff = staff.merge(planta, left_on='Planta', right_on='ID', how='left')


In [221]:
staff.drop(columns={'ID'}, inplace=True)

staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España


In [222]:
len(staff)

4180

Now we are going to upload some staff data to get the number of trainning hours per plant and have a reference of the quantity of empleados we want to generate. This data is aggregated so it is not valid for our analysis

In [223]:
rrhh_plant = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')

rrhh_plant['Año'].unique()

array([2017., 2016., 2018., 2019.,   nan, 2020., 2021.])

In [224]:
# we are getting only 2020 for our analysis

rrhh_plant = rrhh_plant[rrhh_plant['Año']==2020]


In [225]:
rrhh_plant.head()

,ID,Planta,Año,Mes,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Horas Presencia Efectiva Propios,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Horas Formacion General Propios,Empleados Propios - indefinido,Empleados Propios - duración determinada
2615,3819,Barcelona,2020.0,ENE,203.0,28.0,8.0,20.0,29770.30,3138.0,1307.5,15.0,23.0,211.0,189.0,14.0
2616,3820,Burgos,2020.0,ENE,239.0,68.0,28.0,40.0,32625.00,6487.0,4549.3,445.0,348.5,757.0,188.0,38.0
2617,3821,Tarragona,2020.0,ENE,49.0,17.0,5.0,12.0,8352.50,2080.0,862.0,8.0,12.0,111.0,46.0,3.0
2618,3822,Valencia,2020.0,ENE,153.0,28.0,24.0,3.0,22105.20,647.0,3968.3,8.0,6.0,35.5,139.0,2.0
2619,3823,Portugal,2020.0,ENE,99.0,12.0,10.0,2.0,16625.75,384.0,1866.5,80.0,32.5,514.0,72.0,28.0


In [226]:
trainning_hours = rrhh_plant[['Planta','Horas Presencia Efectiva Subcontratados', 'Horas Presencia Efectiva ETTs', 'Horas Formacion Seguridad Propios', 'Horas Formacion Seguridad ETTs']]

In [227]:
trainning_hours.rename(columns={'Planta':'Fabrica'}, inplace=True)
trainning_hours.head(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Fabrica,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
2615,Barcelona,3138.0,1307.5,15.0,23.0


In [228]:
trainning_hours = trainning_hours.groupby('Fabrica').agg('sum').reset_index()

In [229]:
staff = staff.merge(trainning_hours, on='Fabrica', how='left')
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España,25249.0,42484.0,1419.3,830.1
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5


Now we are going to calculate the proportion in 2020 between people and accidents occur

In [230]:
employees = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')
employees = employees[['Año', 'Empleados Propios', 'Empleados Ajenos', 'Empleados ETTS',  'Empleados Subcontrata']]
employees = employees.fillna(0)
employees = employees.groupby(['Año']).agg('sum')
employees.head()

,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata
Año,,,,
0.0,15754.0,3277.0,2490.0,786.0
2016.0,40609.0,9160.0,0.0,0.0
2017.0,43291.0,9794.0,0.0,0.0
2018.0,49865.0,10973.0,7964.0,2993.0
2019.0,50625.0,10527.0,7823.0,2697.0


In [231]:
accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')
accidentes = accidentes[['Año', 'Codigo accidentado']]
accidentes = accidentes.groupby('Año').agg('count').reset_index()
accidentes.rename(columns={'Codigo accidentado':'Accidentes'}, inplace=True)
accidentes.head()

,Año,Accidentes
0,2015.0,202
1,2016.0,1335
2,2017.0,1203
3,2018.0,1532
4,2019.0,1526


In [232]:
employees = employees.merge(accidentes, on='Año', how='inner')
employees.head()

,Año,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Accidentes
0,2016.0,40609.0,9160.0,0.0,0.0,1335
1,2017.0,43291.0,9794.0,0.0,0.0,1203
2,2018.0,49865.0,10973.0,7964.0,2993.0,1532
3,2019.0,50625.0,10527.0,7823.0,2697.0,1526
4,2020.0,65644.0,7114.0,5773.0,1335.0,1122


In [233]:
employees.dtypes

Año                      float64
Empleados Propios        float64
Empleados Ajenos         float64
Empleados ETTS           float64
Empleados Subcontrata    float64
Accidentes                 int64
dtype: object

In [234]:
employees['Total_emp'] = employees['Empleados Propios'] + employees['Empleados Ajenos'] + employees['Empleados ETTS'] + employees['Empleados Subcontrata']

In [235]:
employees = employees[['Año', 'Accidentes', 'Total_emp']]

employees['perc_acc'] = employees['Accidentes'] / employees['Total_emp'] * 100
employees.head()

,Año,Accidentes,Total_emp,perc_acc
0,2016.0,1335,49769.0,2.682393
1,2017.0,1203,53085.0,2.266177
2,2018.0,1532,71795.0,2.133853
3,2019.0,1526,71672.0,2.129144
4,2020.0,1122,79866.0,1.404853


For our analysis we are going to suppose we have a year staff, with all people. 
We would have 79900 people with around 7000 people who have suffered an accident on the past years.

In [236]:
len(staff)

4180

In [237]:
#we add a new column to indetify those people who have suffered an accident in our dataset
staff['Accident'] = 'Yes'
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1,Yes
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España,25249.0,42484.0,1419.3,830.1,Yes
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5,Yes


In [238]:
#plant data we dont have any null values
staff['Planta'].unique()

array([20, 19, 12, 41,  7,  1, 13, 28,  2,  5, 26, 16, 25, 15, 36, 35, 33,
       53, 54,  4, 14, 34, 27, 29, 32, 57,  6, 56, 30, 55, 46, 43, 45, 18,
       44, 63, 47, 84])

In [239]:
##work position

positions = staff['Puesto habitual de trabajo'].unique()
len(positions)

794

In [240]:
import random
import numpy as np 

staff['wk_position'] = np.random.choice(positions, size=len(staff))

In [241]:
staff['wk_position'].unique()

array(['galvanized-hanging-handling', 'Auxiliar Trans 4',
       'Equipo de preparación de montajes',
       'Manual operation on protective rods', ': Operador II',
       'Mantenimiento electrico', 'Red Bud Blanking Line',
       'Desmontaje de la herramienta después de la producción de la línea 1 para colocarla en el almacén de matrices – matriz núm. 26',
       'Titular de Linea', 'operatorof Press', 'EMBALADORA',
       'Montador Prensa 002', '包装packaging', 'EMBALADORA LC6',
       'CARRETILLERO TRANS005', 'embaladora nave 2',
       'Operating the portalcrane', 'carretillero', 'OPERARIO LONGI 3',
       'Packing team member ', 'operator of packaging',
       'Maschinenführer 2 Presse 2', 'Gruista tubos', 'Instandhalter',
       'Ayudante de perfiladora de Dimeco', 'Titular LCL',
       'soldador/gruista', 'Gruista Nave 2 - 3',
       'Operario Entrada Decapado M1', 'Alimentador P3',
       'Chófer/camionero', 'EMBALADORA 2', 'Cizal - Guilhotina',
       'Auxiliares de Logistica\r\

In [242]:
staff.drop(columns={'Puesto habitual de trabajo'}, inplace=True)

In [243]:
#gender

staff.rename(columns={'Género':'Gender'}, inplace=True)


In [244]:
staff['Gender'].unique()

array(['Hombre', nan, 'Mujer'], dtype=object)

In [245]:
staff['Gender'] = staff['Gender'].fillna('Hombre')

In [246]:
len(staff[staff['Gender'].isnull()])

0

In [247]:
staff['Gender'].unique()

array(['Hombre', 'Mujer'], dtype=object)

Checking if we have null values in turno

In [248]:
staff.Turno.isnull().any()

False

Checking for nulls in age:

As we have null values we are goign to reassing the age range based on the proportion of the tables we hav analyzed

In [249]:
staff.Edad.isnull().any()

True

In [250]:
len(staff[staff.Edad.isnull()])

1855

In [251]:
rangos_edad = ['Menor 30', 'Entre 30-50', 'Mayor 50']
weights = {'Hombre':[0.2, 0.44, 0.36], 'Mujer':[0.2,0.55,0.25]}

In [252]:
np.random.seed(0)

def apply_randoms(x):
  key = x['Gender'].iat[0]
  return pd.Series(np.random.choice(rangos_edad, size=len(x), p=weights[key]))

In [253]:
staff['Age'] = staff.groupby('Gender').apply(apply_randoms)\
       .rename('Age').reset_index().drop('level_1', 1)['Age']

In [254]:
staff.drop(columns={'Edad'}, inplace=True)

Checking is there is any row with null at type of accident

In [255]:
staff['Tipo accidente'].isnull().any()

False

Worker type have null values at some rows

In [256]:
staff['Tipo trabajador'].isnull().any()

True

In [257]:
staff['Tipo trabajador'].unique()

array(['ETT', 'Indefinido', 'Duración Determinada',
       'Incidente (no aplica)', 'Subcontratado', 'Prácticas', nan,
       'Cedidos de otras plantas', 'No aplica', 'Indefinido con preaviso'],
      dtype=object)

In [258]:
staff[staff['Tipo trabajador'].isnull()] #As we only have two values null we are going to assign directly 'Indefinido'

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age
2831,HS-NVRR_8597,Turno de mañana,Hombre,13,Incidente,NaN,Leve,Carretilla,Burgos,Blanking,España,25249.0,42484.0,1419.3,830.1,Yes,lider,Menor 30
2832,HS-NVRR_8598,Turno de mañana,Hombre,13,Incidente,NaN,Leve,Carretilla,Burgos,Blanking,España,25249.0,42484.0,1419.3,830.1,Yes,Neutra De Galvanizado,Mayor 50


In [259]:
staff['Tipo trabajador'] = staff['Tipo trabajador'].fillna('Indefinido')

In [260]:
staff['Tipo trabajador'].isnull().any()

False

Potential Severity columns has null values as well

In [261]:
staff[staff['Accident']=='Yes']['Gravedad potencial'].isnull().any()

True

In [262]:
len(staff[(staff['Gravedad potencial'].isnull()) & (staff['Accident']=='Yes')])

1009

In [263]:
## we are going to assign it randomly
staff['Gravedad potencial'].unique()
severity = ['Muy grave', 'Leve', 'Grave']

In [264]:
for i in staff['Gravedad potencial']:

  if str(i) == 'nan':
    staff['Severity'] = np.random.choice(severity, size=len(staff))
  else:
    staff['Severity'] = i


In [265]:
staff['Severity'].isnull().any()

False

In [266]:
staff['Severity'].unique()

array(['Leve', 'Muy grave', 'Grave'], dtype=object)

In [267]:
staff.drop(columns={'Gravedad potencial'}, inplace=True)

In [268]:
staff['Severity'].isnull().any()

False

Aciccent's place/section

In [269]:
staff['Lugar o seccion accidente'].isnull().any()

True

In [270]:
len(staff[staff['Lugar o seccion accidente'].isnull()])

63

In [271]:
staff['Lugar o seccion accidente'].unique()

array(['Células Soldadura', 'Tubos', 'Almacén de Producto Terminado',
       'Blanking', 'Corte Longitudinal', 'Corte Transversal',
       'Almacén de Materia Prima', 'Cizalla', 'Decapado', 'Puente Grúa',
       'Carpinteria', 'Galvanizado Inmersión', nan, 'Maquinaria Auxiliar',
       'Corte Plasma', 'Perfiles abiertos', 'Corte Láser',
       'Almacen productos quimicos', 'Corte Láser 3D',
       'Instalaciones exteriores y auxiliares', 'Mantenimiento', 'Prensa',
       'Almacén producto semielaborado o en producción', 'Carretilla',
       'Sierra', 'Oficinas administrativas', 'Área de embalado',
       'Punzonado', 'Soldadura Manual', 'Limpieza/Prepintado',
       'Galvanizado continuo', 'Perforado', 'Coil to Coil',
       'Tailor Welding', 'Galvanizado ', 'Almacen de recambios',
       'Galvanizado', 'Electrozincado', 'Pintura', 'General / Auxiliar',
       'Perfilado de barrera y postes', 'Skinpass', 'Slitting ',
       'General', 'Press Blanking', 'Laminado', 'Ablation Line',
    

In [272]:
place_acc = ['Células Soldadura', 'Tubos', 'Almacén de Producto Terminado',
       'Blanking', 'Corte Longitudinal', 'Corte Transversal',
       'Almacén de Materia Prima', 'Cizalla', 'Decapado', 'Puente Grúa',
       'Carpinteria', 'Galvanizado Inmersión',  'Maquinaria Auxiliar',
       'Corte Plasma', 'Perfiles abiertos', 'Corte Láser',
       'Almacen productos quimicos', 'Corte Láser 3D',
       'Instalaciones exteriores y auxiliares', 'Mantenimiento', 'Prensa',
       'Almacén producto semielaborado o en producción', 'Carretilla',
       'Sierra', 'Oficinas administrativas', 'Área de embalado',
       'Punzonado', 'Soldadura Manual', 'Limpieza/Prepintado',
       'Galvanizado continuo', 'Perforado', 'Coil to Coil',
       'Tailor Welding', 'Galvanizado ', 'Almacen de recambios',
       'Galvanizado', 'Electrozincado', 'Pintura', 'General / Auxiliar',
       'Perfilado de barrera y postes', 'Skinpass', 'Slitting ',
       'General', 'Press Blanking', 'Laminado', 'Ablation Line',
       'Slitting', 'D-Ring Line', 'Multi Strand']

In [273]:
for i in staff['Lugar o seccion accidente']:

  if str(i) == 'nan':
    staff['Accident_place'] = np.random.choice(place_acc, size=len(staff))
  else:
    staff['Accident_place'] = i

In [274]:
staff['Accident_place'].isnull().any()

False

In [275]:
staff.drop(columns={'Lugar o seccion accidente'}, inplace=True)

There arent any null values in Fabric column, technology and country fields are from fabric so the will be filled as well

In [276]:
staff['Fabrica'].isnull().any()

False

Now we are going to check the columns of efective presence hours

In [277]:
staff['Horas Presencia Efectiva Subcontratados'].isnull().any()

False

In [278]:
staff['Horas Presencia Efectiva ETTs'].isnull().any()

False

In [279]:
staff['Horas Formacion Seguridad Propios'].isnull().any()

False

In [280]:
staff['Horas Formacion Seguridad ETTs'].isnull().any()

False

In [281]:
staff.head()

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place
0,HS-CLMB_2546,Turno de mañana,Hombre,20,Accidente c/b,ETT,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1,Yes,galvanized-hanging-handling,Entre 30-50,Leve,Mantenimiento
1,HS-CEPA_2563,Turno de mañana,Hombre,19,Accidente c/b,Indefinido,Burgos,Recocido,España,25249.0,42484.0,1419.3,830.1,Yes,Auxiliar Trans 4,Mayor 50,Muy grave,Mantenimiento
2,HS-GNVT_2564,Turno de tarde,Hombre,12,Accidente s/b,Duración Determinada,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes,Equipo de preparación de montajes,Entre 30-50,Muy grave,Mantenimiento
3,HS-GNVT_2565,Turno de mañana,Hombre,12,Incidente,ETT,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes,Manual operation on protective rods,Entre 30-50,Grave,Mantenimiento
4,HS-DNGG_2566,Turno de mañana,Hombre,41,Incidente,Incidente (no aplica),Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5,Yes,: Operador II,Entre 30-50,Grave,Mantenimiento


Now we are going to generate the other part of our staff dataframe which contains those people who havent had an accident

In [282]:
len(staff)

4180

In [283]:
employees = rrhh_plant[['Año', 'Mes', 'Empleados Propios', 'Empleados Ajenos','Empleados ETTS','Empleados Subcontrata']]
employees  = employees[employees['Mes']=='ENE']

employees['Total'] = employees['Empleados Propios'] + employees['Empleados Ajenos'] + employees['Empleados ETTS'] + employees['Empleados Subcontrata']

employees = employees.groupby(['Año', 'Mes']).agg('sum')
employees['Total']

Año     Mes
2020.0  ENE    7070.0
Name: Total, dtype: float64

So the number of employees in 2020 7070 and we are going to take that as reference. 

As we are taking 3 years of accidents we are going to have the same proportion on the staff to be able to evaluate it. So our total number of employees in the staff table will be 21.210, 4180 accidents. 

We are going to generate now the 17030 employees who didnt have any accident

In [284]:
staff_without = pd.DataFrame(columns=staff.columns)

staff_without

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place


In [285]:
def create_rndm(values, column):
  staff_without[column] = np.random.choice(values, size=17030)

In [286]:
turno  = ['Turno de mañana', 'Turno de tarde', 'Turno de noche']

create_rndm(turno, 'Turno')

To create the gender columns we are going to apply weights based on the data we had from staff

In [287]:
gender = ['Hombre', 'Mujer']
weights = [0.90,0.1]

np.random.seed(0)

staff_without['Gender'] = random.choices(gender, weights=weights, k=len(staff_without))

In [288]:
planta_list = staff.Planta.unique()
create_rndm(planta_list, 'Planta')

In [289]:
staff_without.head(1)

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place
0,NaN,Turno de tarde,Mujer,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
staff_without.drop(columns={'Fabrica', 'Tecnología', 'Pais'}, inplace=True)

In [291]:
staff_without = staff_without.merge(planta, left_on='Planta', right_on='ID', how='left')
staff_without.head(2)

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place,ID,Fabrica,Tecnología,Pais
0,NaN,Turno de tarde,Mujer,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,Burgos,Skinpass,España
1,NaN,Turno de mañana,Hombre,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,Hiasa,Corte Láser,España


In [292]:
len(staff_without)

17030

In [293]:
tipo_accidente = staff['Tipo accidente'].unique()
create_rndm(tipo_accidente, 'Tipo accidente')

In [294]:
tipo_trabj = staff['Tipo trabajador'].unique()
create_rndm(tipo_trabj, 'Tipo trabajador')

In [295]:
staff_without.drop(columns=['Horas Presencia Efectiva Subcontratados','Horas Presencia Efectiva ETTs', 'Horas Formacion Seguridad Propios', 'Horas Formacion Seguridad ETTs' ], inplace=True)

In [296]:
staff_without = staff_without.merge(trainning_hours, on='Fabrica', how='left')

In [297]:
staff_without.drop(columns={'ID'}, inplace=True)

In [298]:
staff_without['Accident'] = 'No'

In [299]:
pos_wk = staff['wk_position'].unique()

create_rndm(pos_wk, 'wk_position')

In [300]:
rangos_edad = ['Menor 30', 'Entre 30-50', 'Mayor 50']
weights = {'Hombre':[0.2, 0.44, 0.36], 'Mujer':[0.2,0.55,0.25]}

np.random.seed(0)

staff_without['Age'] = staff_without.groupby('Gender').apply(apply_randoms)\
       .rename('Age').reset_index().drop('level_1', 1)['Age']

In [302]:
place = staff['Accident_place'].unique()

create_rndm(place, 'Accident_place')

In [303]:
staff_without.head(2) ##cod accidentado is the only column we are going to leave blank as these employees doestn have any accident

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Accident,wk_position,Age,Severity,Accident_place,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,NaN,Turno de tarde,Mujer,20,Accidente c/b,Incidente (no aplica),No,Auxiliar de Producción,Entre 30-50,NaN,Mantenimiento,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1
1,NaN,Turno de mañana,Hombre,41,Accidente s/b,Indefinido con preaviso,No,poligon production-black area,Mayor 50,NaN,Mantenimiento,Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5


Now we are going to concat both dataframes on a resulting dataframe with length of 21.210

In [304]:
staff = pd.concat([staff, staff_without])
len(staff)

21210

In [305]:
from google.colab import files

staff.to_csv('Total_staff_by_employee.csv', sep=';')
files.download("Total_staff_by_employee.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>